# Python practice 3: feature extraction and classification


Let's start by importing the necessary libraries and loading/creating some example images that we will use throughout this practice.


In [ ]:
# --- Required Imports ---
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float, img_as_ubyte, io, measure, morphology, feature, transform, color, filters
from scipy import ndimage
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import cv2 # OpenCV for some operations such as template matching and moments
import os
from numpy.fft import fft, ifft, fft2, ifft2, fftshift, ifftshift

# --- Helper Plotting Function ---
def plot_many_images_c11_12(images, titles, rows=1, cols=None, cmaps=None, figsize=(15,10), main_title=None):
    num_images = len(images)
    if cols is None:
        cols = (num_images + rows - 1) // rows

    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False)
    axes_flat = axes.ravel()

    if cmaps is None:
        cmaps_list = [None] * num_images
    elif isinstance(cmaps, str):
        cmaps_list = [cmaps] * num_images
    else:
        cmaps_list = list(cmaps)
        if len(cmaps_list) < num_images:
            last_cmap = cmaps_list[-1] if cmaps_list else None
            cmaps_list.extend([last_cmap] * (num_images - len(cmaps_list)))

    for i in range(len(axes_flat)):
        if i < num_images:
            img, title, cmap_val = images[i], titles[i], cmaps_list[i]
            axes_flat[i].imshow(img, cmap=cmap_val if img.ndim==2 or (img.ndim==3 and img.shape[-1]==1) else None)
            axes_flat[i].set_title(title)
            axes_flat[i].axis('off')
        else:
            axes_flat[i].axis('off')

    if main_title:
        fig.suptitle(main_title, fontsize=16)

    plt.tight_layout(rect=[0, 0, 1, 0.96] if main_title else None)
    plt.show()

# --- Create/Load Example Images ---
# Image for contours and descriptors
simple_shape = np.zeros((100, 100), dtype=np.uint8)
simple_shape[20:80, 30:70] = 1 # Rectangle
simple_shape[40:60, 50:90] = 1 # Adds a part for an L shape
simple_shape_labels = measure.label(simple_shape)
simple_shape_props = measure.regionprops(simple_shape_labels)
contour_coords = None
if simple_shape_props: # Get contour of the largest object
    # Find the largest object (if there are multiple disconnected ones)
    largest_obj_idx = np.argmax([prop.area for prop in simple_shape_props])
    contour_coords = measure.find_contours(simple_shape_labels == (largest_obj_idx + 1), 0.5)[0]


# Image for texture and regional moments (we will use the brain one)
try:
    brain_volume = data.brain()
    if brain_volume.ndim == 3:
        slice_idx = brain_volume.shape[0] // 2
        brain_slice_c11 = brain_volume[slice_idx, :, :]
    elif brain_volume.ndim == 2:
        brain_slice_c11 = brain_volume
    else: # Fallback if data.brain() is not as expected
        print("Unexpected format for data.brain(), using data.camera()")
        brain_slice_c11 = data.camera()
except Exception as e:
    print(f"Error loading data.brain(): {e}. Using data.camera().")
    brain_slice_c11 = data.camera()
brain_slice_c11_ubyte = img_as_ubyte(brain_slice_c11)


# Image for template matching
checker_c11 = data.checkerboard()
# Create a template from the image
template_c11 = checker_c11[20:70, 20:70].copy() # A 50x50 template

print("Example images and shapes are ready.")
plot_many_images_c11_12(
    [simple_shape, brain_slice_c11_ubyte, checker_c11, template_c11],
    ["Simple Binary Shape", "Brain Slice (Gray)", "Checkerboard", "Checkerboard Template"],
    2, 2, cmaps=['gray','gray','gray','gray']
)
if contour_coords is not None:
    plt.figure(figsize=(5,5))
    plt.plot(contour_coords[:, 1], contour_coords[:, 0], linewidth=2)
    plt.gca().invert_yaxis() # The image y-axis is inverted
    plt.title("Simple Shape Contour")
    plt.show()

### Interpreting the Results (Initial Setup):

- This block should load the libraries and create/load the example images without errors.
- You will see: a simple binary shape, a grayscale brain slice, a checkerboard, and a small template taken from the checkerboard.
- The contour of the simple shape will also be plotted, which we will use for contour descriptors.


## Module 1: Chapter 11

We will focus on how to represent objects (after segmentation) and describe them quantitatively.


### 1.1. Representation:
**Brief Explanation:**

- Representation: After segmentation, objects can be represented by their external characteristics (contour) or internal characteristics (region).
- Contour Following: Algorithms used to trace the boundary of an object in a binary image.
- Chain Codes: Represent a contour as a sequence of small unit-length directional segments (using, for example, 4 or 8 directions). It is a compact representation and invariant to translation.


**How the Code Works (Simple Chain Code):**

1. Uses `contour_coords` from the simple shape defined in the setup.
2. Computes differences between coordinates of consecutive contour pixels.
3. Maps those differences to a 4-direction chain code (0: right, 1: up, 2: left, 3: down). (For simplicity, we will not implement initial-point or rotation normalization here).


In [ ]:
# 1.1 Chain Codes (Chain Codes)
print("\n--- Module 1.1: Chain Codes ---")

if contour_coords is not None and len(contour_coords) > 1:
    chain_code_4dir = []
    # Contour coordinates are (row, column)
    # (row decreases = up, row increases = down)
    # (column decreases = left, column increases = right)
    # Adjust for image coordinate system (y increases downward)
    # 0: Right (dx=1, dy=0) -> (0, 1) in (col, row)
    # 1: Up    (dx=0, dy=-1) -> (-1, 0) in (col, row)
    # 2: Left (dx=-1, dy=0) -> (0, -1) in (col, row)
    # 3: Down   (dx=0, dy=1) -> (1, 0) in (col, row)

    # find_contours coords are (row, column)
    # dy = diff_row, dx = diff_col
    # 0: Right (d_col=1, d_row=0)
    # 1: Up    (d_col=0, d_row=-1)
    # 2: Left (d_col=-1, d_row=0)
    # 3: Down   (d_col=0, d_row=1)

    for i in range(len(contour_coords) - 1):
        # diff = contour_coords[i+1] - contour_coords[i]
        # Correcting to (row, column)
        # diff_row = contour_coords[i+1, 0] - contour_coords[i, 0]
        # diff_col = contour_coords[i+1, 1] - contour_coords[i, 1]

        # For chain codes, we typically consider nearest neighbors.
        # find_contours may return non-adjacent points on a grid.
        # For a simple example, we will assume points are adjacent
        # or we will compute the predominant direction.
        # A robust chain code implementation requires contour points to be sequential and adjacent on the grid.
        # `measure.find_contours` returns float coordinates, which may not lie perfectly on the grid.
        # For a didactic example, we will round and compute differences.

        pt_atual = np.round(contour_coords[i]).astype(int)
        pt_prox = np.round(contour_coords[i+1]).astype(int)

        diff_row = pt_prox[0] - pt_atual[0]
        diff_col = pt_prox[1] - pt_atual[1]

        code = -1 # Invalid code
        if diff_col == 1 and diff_row == 0: code = 0 # Right
        elif diff_col == 0 and diff_row == -1: code = 1 # Up
        elif diff_col == -1 and diff_row == 0: code = 2 # Left
        elif diff_col == 0 and diff_row == 1: code = 3 # Down
        # For 8-connectivity, we would add diagonals

        if code != -1:
            chain_code_4dir.append(code)
        # Note: If points are not strictly adjacent on the 4-connected grid,
        # this simple logic may fail or skip points.

    print(f"Contour (first 5 points): \n{np.round(contour_coords[:5]).astype(int)}")
    print(f"4-direction Chain Code (first 20 steps): {chain_code_4dir[:20]}")
    print(f"Chain code length: {len(chain_code_4dir)}")

    # Visualize the original contour
    plt.figure(figsize=(6,6))
    plt.plot(contour_coords[:, 1], contour_coords[:, 0], linewidth=2, label='Original Contour')
    plt.scatter(contour_coords[0, 1], contour_coords[0, 0], c='red', s=100, label='Starting Point', zorder=5)
    plt.gca().invert_yaxis()
    plt.title("Simple Shape Contour and Starting Point")
    plt.legend()
    plt.show()
else:
    print("Contour not found for 'simple_shape'. Skip this example.")

**Interpreting the Results (Chain Codes):**

- Contour: The plot should show the boundary of the "Simple Binary Shape" with the starting point marked.
- Chain Code: The printed list of numbers represents the sequence of directions (0-3) used to trace the contour from the starting point. This is the chain code.
- Length: The number of steps in the chain code, which is related to the object's perimeter.


**Exercises:**
1. How would you extend the code to generate an 8-direction chain code? What would the new codes and conditions for `diff_row` and `diff_col` be?


## 1.2. Contour Descriptors

**Brief Explanation:** After obtaining the contour, we can describe it using several measurements.

- Simple Descriptors: Perimeter length, diameter, eccentricity.
- Fourier Descriptors: Represent the contour in the frequency domain. The contour coordinates (x, y) are treated as a complex signal s(k)=x(k)+jy(k). The DFT of s(k) produces the Fourier descriptors. Using a subset of the first coefficients provides a contour approximation and is invariant to some transformations.


**How the Code Works** (Fourier Descriptors):

1. Uses `contour_coords` from the simple shape.
2. Treats contour coordinates as a complex signal: s(k)=column(k)+j⋅row(k).
3. Computes the 1D DFT of s(k) using `fft()`. The result is the Fourier descriptors.
4. To reconstruct/approximate the contour, a limited number of the first descriptors is used (low frequencies, which capture the general shape), and then the IDFT is computed.
5. Displays the original contour and the reconstructed contour with different numbers of descriptors.


In [ ]:
# 1.2 Contour Descriptors (Fourier Descriptors)
print("\n--- Module 1.2: Fourier Descriptors ---")

if contour_coords is not None and len(contour_coords) > 1:
    # Treat the contour as a complex signal: s(k) = x(k) + j*y(k)
    # Where x is the column coordinate and y is the row coordinate
    # In our contour_coords, contour_coords[:, 1] is column (x), contour_coords[:, 0] is row (y)
    sinal_complexo_contorno = contour_coords[:, 1] + 1j * contour_coords[:, 0]

    # Compute Fourier Descriptors (DFT of complex signal)
    descritores_fourier = fft(sinal_complexo_contorno)

    # Function to reconstruct the contour using N_desc descriptors
    def reconstruir_contorno_fourier(descritores, N_desc, tamanho_original):
        if N_desc <= 0 or N_desc > len(descritores):
            N_desc = len(descritores) # Use all if N_desc is invalid

        # Keep the first N_desc/2 and last N_desc/2 coefficients (low frequencies)
        # If N_desc is odd, adjust
        desc_truncados = np.zeros_like(descritores, dtype=complex)

        # Keep DC component and N_desc-1 lowest frequencies (positive and negative)
        # The DFT spectrum is typically [DC, F_pos, ..., F_Nyquist, ..., F_neg]
        # To keep N_desc, we take DC, (N_desc-1)//2 positives, and (N_desc-1)//2 negatives
        # If N_desc is even, N_desc/2 - 1 positives, N_desc/2 negatives
        # If N_desc is odd, (N_desc-1)/2 positives, (N_desc-1)/2 negatives

        # A simpler way to truncate to N_desc components is
        # keep the first N_desc/2 and last N_desc/2 (after DC).
        # But for rotation and scale invariance, more complex descriptor processing is required.
        # Here, we will focus on reconstructing the overall shape.
        # For visual reconstruction, keeping lower frequencies is most important.
        # fftshift places DC at the center. Without shift, DC is at [0].
        # Low frequencies are near DC and at array ends (due to periodicity).

        meio_n_desc = N_desc // 2
        if N_desc == 1: # Only DC
             desc_truncados[0] = descritores[0]
        elif N_desc > 1 :
            desc_truncados[0] = descritores[0] # Keep DC
            # Keep first (N_desc-1)//2 positive frequencies
            desc_truncados[1 : (N_desc -1)//2 + 1] = descritores[1 : (N_desc -1)//2 + 1]
            # Keep last (N_desc-1)//2 negative frequencies (conjugate symmetry)
            # If N_desc is even, N_desc/2 -1 positives and N_desc/2 negatives
            # Or more simply: keep the first N_desc and handle symmetry if invariance is needed.
            # For simple reconstruction:
            desc_truncados[1:meio_n_desc+1] = descritores[1:meio_n_desc+1]
            if N_desc > 1: # Avoid negative index if N_desc=1
                 desc_truncados[-meio_n_desc:] = descritores[-meio_n_desc:]


        contorno_reconst = ifft(desc_truncados)
        # Scale back to original size if IDFT does not do so automatically
        # (numpy fft/ifft preserve "size" in terms of number of points)
        return np.column_stack((contorno_reconst.imag, contorno_reconst.real)) # (row, column)

    # Reconstruct with different numbers of descriptors
    N_desc_10 = 10
    N_desc_30 = 30
    N_desc_todos = len(descritores_fourier)

    contorno_rec_10 = reconstruir_contorno_fourier(descritores_fourier, N_desc_10, len(sinal_complexo_contorno))
    contorno_rec_30 = reconstruir_contorno_fourier(descritores_fourier, N_desc_30, len(sinal_complexo_contorno))
    contorno_rec_todos = reconstruir_contorno_fourier(descritores_fourier, N_desc_todos, len(sinal_complexo_contorno))

    # Visualization
    fig, axs = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)
    fig.suptitle("Fourier Descriptors and Contour Reconstruction", fontsize=16)

    axs[0].plot(contour_coords[:, 1], contour_coords[:, 0], linewidth=2)
    axs[0].set_title('Original Contour')

    axs[1].plot(contorno_rec_10[:, 1], contorno_rec_10[:, 0], linewidth=2)
    axs[1].set_title(f'Reconstructed ({N_desc_10} Descriptors)')

    axs[2].plot(contorno_rec_30[:, 1], contorno_rec_30[:, 0], linewidth=2)
    axs[2].set_title(f'Reconstructed ({N_desc_30} Descriptors)')

    axs[3].plot(contorno_rec_todos[:, 1], contorno_rec_todos[:, 0], linewidth=2)
    axs[3].set_title(f'Reconstructed (All {N_desc_todos} Desc.)')

    for ax in axs:
        ax.invert_yaxis() # Adjust y-axis orientation
        ax.set_aspect('equal', adjustable='box')
    plt.show()
else:
    print("Contour not found or too small. Skip the Fourier Descriptor example.")

**Interpreting the Results (Fourier Descriptors):**

* Original Contour: The original shape.
* Reconstructed (10 Descriptors): Using only the first 10 Fourier descriptors (which represent the lowest-frequency components of the contour shape), the reconstruction will be a smoothed, approximate version of the original shape, capturing general characteristics but losing fine details.
* Reconstructed (30 Descriptors): With more descriptors, the reconstruction will be more faithful to the original shape, including more details.
* Reconstructed (All Descriptors): Should be (almost) identical to the original contour.

**Exercises**

1. Try different values for `N_desc` in the `reconstruir_contorno_fourier` function. What is the minimum number of descriptors that still provides a visually recognizable representation of the original shape?
2. Fourier descriptors are translation invariant. To make them scale- and rotation-invariant, they are usually further processed (e.g., using only descriptor magnitudes or normalizing by the first non-zero descriptor). Briefly research how rotation and scale invariance can be achieved. (This is more conceptual).


## 1.3. Regional Descriptors

**Brief Explanation:** They describe an entire region (not only its contour).

* Topological Descriptors: E.g., Euler number (number of objects - number of holes).
* Texture: Measures of smoothness, roughness, and regularity. Statistical (histogram, GLCM), structural, and spectral approaches. We will focus on some simple statistics.
* Moments: Geometric and invariant moments (Hu moments) describe region shape and are invariant to translation, scale, and rotation.

**How the Code Works:**

1. Uses `brain_slice_c11_ubyte` and `simple_shape` (filled).
1. Euler Number: `measure.euler_number()`.
1. Texture (Simple Statistics): Computes mean, standard deviation, smoothness (based on variance), uniformity, and entropy from the histogram of a region in the brain image.
1. Hu Moments: `cv2.moments()` computes spatial moments. `cv2.HuMoments()` computes the 7 Hu moments from spatial moments. These are applied to the filled simple shape.


In [ ]:
from skimage.exposure import rescale_intensity, histogram

# 1.3 Regional Descriptors
print("\n--- Module 1.3: Regional Descriptors ---")

if 'brain_slice_c11_ubyte' in globals() and 'simple_shape' in globals():
    # --- a) Topological Descriptors (Euler Number) ---
    # For Euler Number, we need a clean binary image.
    # Use 'simple_shape' and ensure it is filled.
    forma_preenchida_euler = ndimage.binary_fill_holes(simple_shape > 0)
    # euler_number counts objects - holes. For a solid shape, it should be 1.
    # For skimage < 0.19, connectivity is an argument. For >= 0.19, it is not.
    try:
        num_euler = measure.euler_number(forma_preenchida_euler, connectivity=2)
    except TypeError: # Older versions may not have connectivity or may handle it differently
         try:
            num_euler = measure.euler_number(forma_preenchida_euler) # Try without it
         except Exception as e_euler:
            print(f"Error calculating Euler number: {e_euler}")
            num_euler = "Erro"

    print(f"Euler number for filled 'simple_shape': {num_euler}")

    # --- b) Texture (Simple Statistical Descriptors) ---
    # Get a sub-region of the brain image for texture analysis
    # (e.g., a region that appears textured)
    regiao_textura = brain_slice_c11_ubyte[100:150, 100:150]
    if regiao_textura.size == 0: # Fallback if crop is empty
        regiao_textura = brain_slice_c11_ubyte[0:50,0:50]

    if regiao_textura.size > 0:
        media_textura = np.mean(regiao_textura)
        std_textura = np.std(regiao_textura)
        # Smoothness (related to variance) R = 1 - 1/(1+sigma^2)
        # Normalize std to [0,1] if image is in [0,255]
        std_norm_textura = std_textura / 255.0
        suavidade_textura = 1 - (1 / (1 + std_norm_textura**2))

        # Histogram Uniformity and Entropy
        hist_textura, _ = histogram(regiao_textura, nbins=256, source_range='image')
        prob_textura = hist_textura / np.sum(hist_textura) # Normalize to make it a PDF
        uniformidade_textura = np.sum(prob_textura**2)
        entropia_textura = -np.sum(prob_textura * np.log2(prob_textura + 1e-8)) # Add epsilon to avoid log(0)

        print(f"\nTexture Descriptors for selected region:")
        print(f"  Mean: {media_textura:.2f}")
        print(f"  Standard Deviation: {std_textura:.2f}")
        print(f"  Smoothness (R): {suavidade_textura:.4f}")
        print(f"  Uniformity: {uniformidade_textura:.4f}")
        print(f"  Entropy: {entropia_textura:.4f}")
    else:
        print("Texture region is empty.")

    # --- c) Geometric and Hu Invariant Moments ---
    # Use OpenCV to compute moments. Requires ubyte image and single contour.
    # Input image for cv2.moments must be binary (0 or 255) or grayscale.
    # Use 'forma_preenchida_euler', which is binary (0 or 1), convert to ubyte.
    forma_para_momentos_ubyte = img_as_ubyte(forma_preenchida_euler)

    momentos_espaciais = cv2.moments(forma_para_momentos_ubyte)
    # print(f"\nSpatial Moments (dictionary): {momentos_espaciais}")

    # Compute Hu Moments (invariant to translation, scale, rotation)
    momentos_hu = cv2.HuMoments(momentos_espaciais).flatten() # flatten to 1D array
    # Apply log for better visualization/comparison of magnitude orders
    momentos_hu_log = -np.sign(momentos_hu) * np.log10(np.abs(momentos_hu) + 1e-8)


    print(f"\nHu Moments (log-transformed):")
    for i, mom_hu in enumerate(momentos_hu_log):
        print(f"  hu[{i+1}]: {mom_hu:.4f}")

    # Visualization
    plot_many_images_c11_12([forma_preenchida_euler, regiao_textura],
                           [f"Shape for Euler ({num_euler}) and Moments", "Region for Texture"],
                           1, 2, cmaps=['gray','gray'], figsize=(10,5),
                           main_title="Regional Descriptors")
else:
    print("Images 'brain_slice_c11_ubyte' or 'simple_shape' are not defined.")

**Interpreting the Results (Regional Descriptors):**

- Euler Number: For `forma_preenchida_euler` (a solid object), the Euler number should be 1 (1 object - 0 holes).

- Mean: Average gray level of the region.
- Standard Deviation: Measure of gray-level dispersion (local contrast).
- Smoothness (R): Close to 1 for smooth regions (low variance), close to 0 for less smooth regions.
- Uniformity: High for regions with little intensity variation (histogram with a few tall peaks).
- Entropy: High for regions with a more spread-out histogram (complexity/randomness).


- Hu Moments: The 7 printed values are Hu moments (log-transformed for easier visualization, since their magnitudes can vary a lot). They characterize object shape. If you rotate, scale, or translate `forma_preenchida_euler` and recompute Hu moments, they should remain (approximately) the same.


**Exercise:**

1. Texture: Select a different region of the brain image (`brain_slice_c11_ubyte`) with a visually distinct texture from the first one (e.g., a darker and more homogeneous region vs. a brighter and more textured one). Recompute and compare statistical texture descriptors. Do they reflect the visual differences?
2. Hu Moments:
- Create a new simple binary shape (e.g., a circle or triangle).
- Compute its Hu moments.
- Now rotate or slightly scale this new shape (using `skimage.transform.rotate` or `skimage.transform.rescale`) and recompute Hu moments. Are they truly invariant? (Small variations may occur due to discretization).


## Chapter 12 - Object Recognition


We will focus on methods based on theoretical decision principles: template matching and an introduction to statistical classifiers and neural networks.


### 2.1. Patterns, Classes, and Template Matching

**Brief Explanation:**

- Object Recognition: Assigning a label (class) to an object based on its descriptors (features).
- Patterns and Classes: A pattern is an arrangement of descriptors. A class is a set of patterns with common properties.
- Template Matching: A simple method to find occurrences of a pattern (template) in a larger image. The template is slid over the image, and a similarity (or difference) measure is computed at each position.


**How the Code Works (Template Matching):**

1. Uses `checker_c11` as the main image and `template_c11` (a checkerboard crop) as the template.
1. `feature.match_template(image, template, pad_input=True)`: Computes normalized correlation response (or normalized squared difference, depending on internals, but the result is a similarity measure) between the template and all image windows.
* `pad_input=True`: Adds padding to the image so the template can be matched at the borders.
3. Find the Best Match: `np.unravel_index(np.argmax(resultado_matching), resultado_matching.shape)` finds the coordinates (row, column) where matching response (similarity) is maximum.
4. Visualization: Shows the image, template, and image with a rectangle drawn at the best match.


In [ ]:
# 2.1 Template Matching
print("\n--- Module 2.1: Template Matching ---")

if 'checker_c11' in globals() and 'template_c11' in globals():
    # The image and template must be grayscale for skimage match_template
    # If they were colored, we would convert them or perform channel-wise matching.
    # checker_c11 and template_c11 are already grayscale (or can be converted if needed).
    # Ensure they are float for match_template
    img_for_match = img_as_float(checker_c11)
    template_for_match = img_as_float(template_c11)

    # Perform template matching
    # The function returns a similarity map
    matching_result = feature.match_template(img_for_match, template_for_match, pad_input=True)

    # Find the best match position (maximum value in the similarity map)
    ij = np.unravel_index(np.argmax(matching_result), matching_result.shape)
    x_match, y_match = ij[::-1] # Coordinates (column, row) of the top-left corner of the best match

    # Template dimensions
    h_temp, w_temp = template_for_match.shape

    # Create a figure for drawing
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(checker_c11, cmap='gray')
    ax.set_title('Template Matching Result')
    ax.axis('off')

    # Create a rectangle to highlight the best match
    # match_template result may place the "hot spot" at the corner or center of the template.
    # For `pad_input=True`, the output has the same size as the image and `ij` is the top-left corner.
    rect = plt.Rectangle((x_match, y_match), w_temp, h_temp, edgecolor='r', facecolor='none', linewidth=2)
    ax.add_patch(rect)
    plt.show()

    print(f"Template found at corner position (column, row): ({x_match}, {y_match})")

    plot_many_images_c11_12([checker_c11, template_c11, matching_result],
                           ["Main Image", "Template", "Similarity Map (Matching Result)"],
                           1,3, cmaps=['gray','gray','viridis'], figsize=(15,5))
else:
    print("Images 'checker_c11' or 'template_c11' are not defined.")

**Interpreting the Results (Template Matching):**

- Main Image and Template: The input images.
- Similarity Map: An image where each pixel represents how similar that pixel's neighborhood is to the template. Bright points indicate high similarity.
- Visual Result (with rectangle): The main image with a red rectangle drawn around the region identified by the algorithm as the best match for the template.


**Exercise:**

1. Create a new `template_novo` from a different part of `checker_c11`. Does the algorithm still find it?
1. Try using a template that is not present in `checker_c11` (you can create a small random NumPy array or load another small image). What happens to the "similarity map" and the "best match" found?
1. If you rotate or scale the template, will `skimage`'s `match_template` (which is based on normalized correlation) still work well?


## 2.2. Optimal Statistical Classifiers (Bayes)

**Brief Explanation:** Bayesian decision theory provides a framework for building classifiers that minimize error probability (Bayes classifier).

Given an object feature vector **x**, we want to assign it to class ω_{i} that maximizes posterior probability P(ω_{i}∣x).
By Bayes' theorem: P(ω_{i}∣x)=p(x∣ω_{i})P(ω_{i})/p(x), where:
- p(x∣ω_{i}): Conditional probability density of features (likelihood).
- P(ω_{i}): Prior probability of class ω_{i}.
- p(x): Evidence (normalization constant).

Decision: Choose ω_{i} if p(x∣ω_{i})P(ω_{i})>p(x∣ω_{j})P(ω_{j}) for every j!=i.

If PDFs p(x∣ω_{i}) are Gaussian, decision surfaces can be linear or quadratic.


**How the Code Works (Simplified Pixel Classification Example):**
1. Generate Synthetic Data: Creates two clouds of 2D points (representing features of two pixel classes, e.g., intensity and local texture) that follow Gaussian distributions with different means and covariances.
1. Train a Gaussian Bayesian Classifier: `sklearn.naive_bayes.GaussianNB` is a simple classifier that assumes independence between features (naive) and Gaussian distribution for each feature within each class.
1. Create a Grid to Visualize the Decision: Generates a grid of points in feature space.
1. Predict Classes on the Grid: The trained classifier predicts the class for each grid point.
1. Visualization: Plots the original training data and classifier decision region.


In [ ]:
# 2.2 Simple Gaussian Bayesian Classifier
print("\n--- Module 2.2: Gaussian Bayesian Classifier (Simple) ---")
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_blobs # To generate synthetic data

# 1. Generate synthetic data for two Gaussian-distributed classes
#    Each "sample" can represent a pixel described by 2 features
n_amostras_bayes = 300
centros_bayes = [(-1, -1), (2, 2)] # Means das duas classes
std_bayes = 0.8 # Class standard deviation
X_bayes, y_bayes = make_blobs(n_samples=n_amostras_bayes, centers=centros_bayes,
                              cluster_std=std_bayes, random_state=42)

# 2. Train a Gaussian Naive Bayes classifier
gnb = GaussianNB()
gnb.fit(X_bayes, y_bayes)

# 3. Create a grid to visualize decision boundary
x_min_bayes, x_max_bayes = X_bayes[:, 0].min() - 1, X_bayes[:, 0].max() + 1
y_min_bayes, y_max_bayes = X_bayes[:, 1].min() - 1, X_bayes[:, 1].max() + 1
xx_bayes, yy_bayes = np.meshgrid(np.linspace(x_min_bayes, x_max_bayes, 100),
                                 np.linspace(y_min_bayes, y_max_bayes, 100))

# 4. Predict classes for each grid point
Z_bayes = gnb.predict(np.c_[xx_bayes.ravel(), yy_bayes.ravel()])
Z_bayes = Z_bayes.reshape(xx_bayes.shape)

# 5. Visualization
plt.figure(figsize=(8, 6))
plt.contourf(xx_bayes, yy_bayes, Z_bayes, alpha=0.4, cmap='coolwarm')
plt.scatter(X_bayes[:, 0], X_bayes[:, 1], c=y_bayes, s=20, edgecolor='k', cmap='coolwarm')
plt.title("Gaussian Naive Bayes Classifier - Decision Regions")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid(True)
plt.show()

# Image application example (conceptual, since we would train with real data)
# Take two regions from the brain image representing two classes (e.g., white matter, gray matter)
if 'brain_slice_c11_ubyte' in globals():
    img_cerebro_bayes = brain_slice_c11_ubyte
    # Very simplified example: use intensity as the only feature
    # We could take samples from two regions and their intensities
    # Region 1 (e.g., darker)
    # amostras_r1 = img_cerebro_bayes[100:120, 100:120].ravel()
    # Region 2 (e.g., brighter)
    # amostras_r2 = img_cerebro_bayes[150:170, 150:170].ravel()
    # ... train gnb with these samples ...
    # ... then classify all pixels in image_cerebro_bayes ...
    print("To apply this to a real image, we would need to extract features from known regions,")
    print("train the classifier, and then apply it to the entire image.")
else:
    print("Brain image not available for conceptual Bayes example.")

**Interpreting the Results (Simple Bayes):**

- The plot shows two point clouds representing samples from two classes (blue and red).
- The colored background area (light blue and light red) represents decision regions learned by the Gaussian Naive Bayes classifier. If a new sample falls in the light blue region, it will be classified as class 0 (blue); if it falls in the light red region, class 1 (red).
- The line (or curve) separating the two colored regions is the decision boundary. For Gaussian Naive Bayes with class-specific covariances (default), this boundary can be quadratic. If covariances were assumed equal, it would be linear.



**Exercise:**

1. In `make_blobs`, change `centros_bayes` so that classes are closer together or more overlapping. Re-run the code. How does the decision boundary change? Is separation still good?
1. Change `std_bayes` (cluster standard deviation) to a larger value (e.g., 1.5). How does this affect class overlap and decision boundary?
1. If you had a medical image and wanted to segment two tissue types with this classifier, what pixel features (besides intensity) could you use to form feature vector x?



## 2.3. Artificial Neural Networks (Introduction)

**Brief Explanation:** Artificial Neural Networks (ANNs), especially Multi-Layer Perceptrons (MLPs), are powerful classifiers that can learn complex, non-linear decision boundaries.

- Basic Structure: Input layer (features), one or more hidden layers (with neurons and non-linear activation functions such as ReLU or sigmoid), and an output layer (classes).
- Training: The process of adjusting connection weights between neurons using an algorithm such as backpropagation to minimize an error on a training dataset.


**How the Code Works (Simple MLP with scikit-learn):**

1. Data: Reuses synthetic data `X_bayes`, `y_bayes` from the previous example.
1. Train/Test Split: `train_test_split` divides data into training and test sets to evaluate model generalization.
1. Feature Scaling: `StandardScaler` normalizes features (mean 0, variance 1). This is often important for good neural network performance.

1. MLP Creation and Training:

`MLPClassifier(hidden_layer_sizes=(10, 5), ...)`: Defines an MLP with two hidden layers, the first with 10 neurons and the second with 5.
- `activation='relu'`: ReLU activation function.
- `solver='adam'`: Optimizer for weight adjustment.
- `max_iter`: Maximum number of training epochs.
- `random_state`: For reproducibility.
- `mlp.fit(X_train_scaled, y_train)`: Trains the model.

5. Prediction and Evaluation:
`mlp.predict(X_test_scaled)`: Makes predictions on the test set.
`accuracy_score()`: Computes accuracy.
6. Visualization: Plots data and decision boundary learned by the MLP.


In [ ]:
# 2.3 Artificial Neural Networks (MLP Simples com scikit-learn)
print("\n--- Module 2.3: Neural Networks (Basic MLP) ---")

if 'X_bayes' in globals() and 'y_bayes' in globals():
    # 1. Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_bayes, y_bayes, test_size=0.3, random_state=42)

    # 2. Scale features (important for Neural Networks)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Create and Train the MLPClassifier
    #    hidden_layer_sizes: tuple, e.g., (100,) for one hidden layer with 100 neurons
    #                          e.g., (50, 20) for two hidden layers
    mlp = MLPClassifier(hidden_layer_sizes=(10, 5), # Two hidden layers
                        activation='relu',        # Activation function
                        solver='adam',            # Optimizer
                        max_iter=1000,            # Maximum training iterations
                        random_state=42,
                        early_stopping=True,      # Stop early if no improvement
                        n_iter_no_change=20)      # Patience for early stopping

    print("Training MLP...")
    mlp.fit(X_train_scaled, y_train)
    print("MLP training completed.")

    # 4. Make predictions and evaluate
    y_pred_mlp = mlp.predict(X_test_scaled)
    acuracia_mlp = accuracy_score(y_test, y_pred_mlp)
    print(f"MLP accuracy on test set: {acuracia_mlp:.4f}")

    # 5. Visualize decision boundary (similar to Bayes)
    # Recompute grid based on scaled data for correct plotting
    # Or, more simply, use the unscaled grid and transform grid points for prediction
    xx_mlp_plot, yy_mlp_plot = xx_bayes, yy_bayes # Use the same grid as Bayes example

    # To predict on the grid, we need to scale it the same way as training data
    grid_points_flat = np.c_[xx_mlp_plot.ravel(), yy_mlp_plot.ravel()]
    grid_points_flat_scaled = scaler.transform(grid_points_flat) # Use the same trained scaler
    Z_mlp = mlp.predict(grid_points_flat_scaled)
    Z_mlp = Z_mlp.reshape(xx_mlp_plot.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(xx_mlp_plot, yy_mlp_plot, Z_mlp, alpha=0.4, cmap='coolwarm')
    # Plot original (unscaled) data for correct axes
    plt.scatter(X_bayes[:, 0], X_bayes[:, 1], c=y_bayes, s=20, edgecolor='k', cmap='coolwarm')
    plt.title(f"MLP Classifier - Decision Regions (Accuracy: {acuracia_mlp:.2f})")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.grid(True)
    plt.show()
else:
    print("Data 'X_bayes', 'y_bayes' not defined. Skip this example.")

**Interpreting the Results (Simple MLP):**

- Accuracy: The printed value indicates the percentage of samples in the test set that were correctly classified by the trained MLP.
- Decision Regions: The plot shows input data and decision regions learned by the MLP. Unlike Naive Bayes (which assumes specific distributions and independence), the MLP can learn more complex and non-linear decision boundaries, which may be visible if data clusters are not linearly separable or have complex shapes. The boundary can be more "flexible."


**Exercise:**

1. Change the MLP architecture in `hidden_layer_sizes`:
- Try a single hidden layer with more neurons: `(50,)`.
- Try more layers or more neurons: `(20, 10, 5)`.
- How does architecture affect accuracy and decision boundary complexity? (You may need to increase `max_iter` for larger networks to converge).
2. Change activation function to `'tanh'` or `'logistic'`. Is there any noticeable change in decision boundary or accuracy for this simple problem?
3. To use an MLP to segment a medical image (e.g., tumor vs. healthy tissue), which features (descriptors from Chapter 11) would you extract from image regions to feed into the network input layer?
